In [ ]:
from xdrlib import ConversionError

"""
#6.7 Recap (05:17)
#6.8 Stuff LCEL Chain (10:13)

1. File들을 Load 하기.
 - TextLoader, UnstructuredFileLoader
2. Split (거대한 단일 document 보다 작은 여러개를 LLM 전달할 때 검색 성능이 좋다
3. Ebmedding : text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현한거
 - O

"""

In [12]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)
cache_dir = LocalFileStore("./.cache/")

# chunk_size : 얼마나 큰 덩어리로 나눌지 결정할 수 있음
# chunk_overlap : 문장이나 문단을 분할할 때 앞 조각 일부분을 가져옴
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/현진건-운수_좋은_날.txt",encoding="utf-8")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)


In [14]:
memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=20,
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [16]:
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    print(result)
    memory.save_context({"input": question}, {"output": result.content})

In [17]:
invoke_chain("김첨지의 직업은 무엇?")

content='김첨지는 구걸하는 거지나 무엇같이 연해연방인 것 같은 기생 퇴물로 묘사되어 있습니다.'


In [ ]:
load_memory({})